<a href="https://colab.research.google.com/github/zambbo/CNN-DialectDetector/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
import os
import re
from matplotlib import pyplot as plt
from glob import glob
import numpy as np
import pickle
from tqdm import tqdm
import time
from sklearn.preprocessing import MinMaxScaler

In [5]:
index2region={0:'gangwon', 1:'gyeongsang', 2:'jeonla', 3:'chungcheong', 4:'jeju'}
region2index = {v:k for k,v in index2region.items()}
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [6]:
# 데이터 셋 구성 (small dataset)
dataset_dir = '/content/drive/MyDrive/DialectDataset/small_dataset/'

In [7]:
region_dir = glob(dataset_dir)
region_dir

['/content/drive/MyDrive/DialectDataset/small_dataset/']

In [8]:
for k, v in index2region.items():
    exec(f"{v}_dirs = glob(dataset_dir+'*_{v}/*')")
jeonla_dirs

['/content/drive/MyDrive/DialectDataset/small_dataset/preprocessed_jeonla/DJDD20000014',
 '/content/drive/MyDrive/DialectDataset/small_dataset/preprocessed_jeonla/DJDD20000012',
 '/content/drive/MyDrive/DialectDataset/small_dataset/preprocessed_jeonla/DJDD20000019',
 '/content/drive/MyDrive/DialectDataset/small_dataset/preprocessed_jeonla/DJDD20000006',
 '/content/drive/MyDrive/DialectDataset/small_dataset/preprocessed_jeonla/DJDD20000015',
 '/content/drive/MyDrive/DialectDataset/small_dataset/preprocessed_jeonla/DJDD20000032',
 '/content/drive/MyDrive/DialectDataset/small_dataset/preprocessed_jeonla/DJDD20000027',
 '/content/drive/MyDrive/DialectDataset/small_dataset/preprocessed_jeonla/DJDD20000018',
 '/content/drive/MyDrive/DialectDataset/small_dataset/preprocessed_jeonla/DJDD20000005',
 '/content/drive/MyDrive/DialectDataset/small_dataset/preprocessed_jeonla/DJDD20000024']

In [9]:
def make_tuple_data(dirs, max_num):
    for i, region_dir in enumerate(dirs):
        if i>=max_num:break
        spectro_path = glob(region_dir+'/*_spectro.pickle')[0]
        mfcc_path = glob(region_dir+'/*_mfcc.pickle')[0]
        chroma_path = glob(region_dir+'/*_chroma.pickle')[0]
        
        with open(spectro_path, "rb") as f:
            spectro = pickle.load(f)
        with open(mfcc_path, "rb") as f:
            mfcc = pickle.load(f)
        with open(chroma_path, "rb") as f:
            chroma = pickle.load(f)

        if i == 0:
            spectro_data = spectro
            mfcc_data = mfcc
            chroma_data = chroma
        else:
            spectro_data = np.concatenate([spectro_data,spectro], axis=0)
            mfcc_data = np.concatenate([mfcc_data,mfcc], axis=0)
            chroma_data = np.concatenate([chroma_data,chroma], axis=0)
        
    r_data = [(s,m,c) for s,m,c in zip(spectro_data,mfcc_data,chroma_data)]
        
    return r_data
jeonla_data = make_tuple_data(jeonla_dirs, 3)
chungcheong_data = make_tuple_data(chungcheong_dirs, 3)
gyeongsang_data = make_tuple_data(gyeongsang_dirs, 3)
jeju_data = make_tuple_data(jeju_dirs, 3)
gangwon_data = make_tuple_data(gangwon_dirs, 3)

print("data num: ", len(jeonla_data))
print("tuple size", len(jeonla_data[0]))
print("spec shape", jeonla_data[0][0].shape)

data num:  322
tuple size 3
spec shape (201, 501)


In [10]:
jeonla_data_l = []
for data in jeonla_data:
    y = [0,0,0,0,0]
    y[region2index['jeonla']] = 1
    jeonla_data_l.append((data,y))

chungcheong_data_l = []
for data in chungcheong_data:
    y = [0,0,0,0,0]
    y[region2index['chungcheong']] = 1
    chungcheong_data_l.append((data,y))

gyeongsang_data_l = []
for data in gyeongsang_data:
    y = [0,0,0,0,0]
    y[region2index['gyeongsang']] = 1
    gyeongsang_data_l.append((data,y))

jeju_data_l = []
for data in jeju_data:
    y = [0,0,0,0,0]
    y[region2index['jeju']] = 1
    jeju_data_l.append((data,y))

gangwon_data_l = []
for data in gangwon_data:
    y = [0,0,0,0,0]
    y[region2index['gangwon']] = 1
    gangwon_data_l.append((data,y))

In [11]:
datasumup = np.concatenate([jeonla_data_l, chungcheong_data_l, gangwon_data_l, jeju_data_l, gyeongsang_data_l], axis=0)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [12]:
print(len(jeonla_data))
print(len(jeonla_data[0]))
print(len(jeonla_data[0][0]))
print(jeonla_data[0][0][0].shape)
print(jeonla_data[0][1])

322
3
201
(501,)
[[-803.2581 -803.2581 -803.2581 ... -803.2581 -803.2581 -803.2581]
 [   0.        0.        0.     ...    0.        0.        0.    ]
 [   0.        0.        0.     ...    0.        0.        0.    ]
 ...
 [   0.        0.        0.     ...    0.        0.        0.    ]
 [   0.        0.        0.     ...    0.        0.        0.    ]
 [   0.        0.        0.     ...    0.        0.        0.    ]]


In [37]:
class MultiModalDataset(Dataset):

    def __init__(self, data):

        self.data = data

    def __getitem__(self, idx):
        datas, label = self.data[idx]
        spec, mfcc, chroma = datas

        spec, mfcc, chroma = torch.tensor(spec, dtype=torch.float32), torch.tensor(mfcc, dtype=torch.float32), torch.tensor(chroma, dtype=torch.float32)
        spec, mfcc, chroma = spec.unsqueeze(0), mfcc.unsqueeze(0), chroma.unsqueeze(0)
        label = torch.tensor(label, dtype=torch.float32)
        data = (spec, mfcc, chroma)
        return data, label
    
    def __len__(self):
        return len(self.data)

In [38]:
dataset = MultiModalDataset(datasumup)
len(dataset)

1285

In [39]:
class BasicBlock(nn.Module):

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()

        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(3,3), stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.seq1 = nn.Sequential(self.conv1, self.bn1, self.relu)
        self.conv2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=(3,3), stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.seq2 = nn.Sequential(self.conv2, self.bn2)
        
        self.down_flag = False
        if in_channels != out_channels: self.down_flag = True

        self.downsample = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(1,1), stride=2, padding=0, bias=False)
    
    def forward(self, x):
        #print(x.shape)
        y = self.seq1(x)
        #print(y.shape)
        y = self.seq2(y)
        #print(y.shape)

        if self.down_flag:
            x = self.downsample(x)
        
        y = self.relu(y)
        #print(x.shape)
        #print(y.shape)
        y = y + x

        return y
        

In [40]:
class ResNet18(nn.Module):

    def __init__(self, in_channels, output_dim=1024, model_type='spec'):
        super(ResNet18, self).__init__()

        self.relu = nn.ReLU()

        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=64, kernel_size=(7,7), stride=2, padding=3)
        self.BN1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=(3,3), stride=2, padding=1)

        self.seq1 = nn.Sequential(self.conv1, self.BN1, self.pool1)

        self.seq2 = nn.Sequential(BasicBlock(64,64), BasicBlock(64,64))
        self.seq3 = nn.Sequential(BasicBlock(64,64), BasicBlock(64, 128, stride=2))
        self.seq4 = nn.Sequential(BasicBlock(128,128), BasicBlock(128,128))
        self.seq5 = nn.Sequential(BasicBlock(128,128), BasicBlock(128,256,stride=2))

        if model_type=='spec':
            self.fc1 = nn.Linear(256*13*32, output_dim)
        elif model_type=='mfcc':
            self.fc1 = nn.Linear(256*7*32, output_dim)
        elif model_type=='chroma':
            self.fc1 = nn.Linear(256*1*32, output_dim)


        self.lastlayer = nn.Sequential(self.fc1, self.relu)

    def forward(self, x):
        y = self.seq1(x)
        y = self.seq2(y)
        y = self.seq3(y)
        y = self.seq4(y)
        y = self.seq5(y)
        y = y.view(y.shape[0],-1)
        y = self.lastlayer(y)

        return y



In [41]:
class MultiModalDialectClassifier(nn.Module):

    def __init__(self, hidden_dim=1024, out_dim=5, learning_rate=0.01, best_model_save_path="./best_model.pt"):
        super(MultiModalDialectClassifier, self).__init__()

        self.best_model_save_path = best_model_save_path
        self.spec_res = ResNet18(1, model_type='spec')
        self.mfcc_res = ResNet18(1, model_type='mfcc')
        self.chroma_res = ResNet18(1, model_type='chroma')
        

        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(1024*3, 512)
        self.fc2 = nn.Linear(512,out_dim)
        self.lastlayer = nn.Sequential(self.fc1, self.relu, self.fc2)

        self.loss_f = nn.CrossEntropyLoss()
        self.optimizer = optim.AdamW(self.parameters(), lr=learning_rate)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        spec_x, mfcc_x, chroma_x = x

        spec_y = self.spec_res(spec_x)
        mfcc_y = self.mfcc_res(mfcc_x)
        chroma_y = self.chroma_res(chroma_x)

        y = torch.cat([spec_y, mfcc_y, chroma_y], dim=1)
        y = y.view(y.shape[0], -1)
        
        y = self.lastlayer(y)
        y = self.softmax(y)
        #print(y.shape)
        return y
    
    def train_(self, train_loader, val_loader, learning_rate, epochs, device):
        self.train_accuracy = []
        self.train_loss = []
        self.val_accuracy = []
        self.val_loss = []
        best_epoch = -1
        best_acc = -1 
        
        for epoch in range(1, epochs+1):
            total = 0
            correct = 0
            start_time = time.time()
            epoch_loss = 0.0
            epoch_acc = 0.0
            self.train()

            for batch_idx, (batch_data, batch_label) in enumerate(tqdm(train_loader)):
                
                spec, mfcc, chroma = batch_data
                spec, mfcc, chroma = spec.to(device), mfcc.to(device), chroma.to(device)
                batch_data = (spec, mfcc, chroma)
                batch_label = batch_label.to(device)

                self.optimizer.zero_grad()

                pred = self.forward(batch_data) # (batch_size, 5)
                loss = self.loss_f(pred, batch_label)
                loss.backward()
                self.optimizer.step()

                epoch_loss += loss.item()

                _, pred_indices = torch.max(pred, axis=1)
                total += batch_data[0].shape[0]
                batch_label = torch.max(batch_label, axis=1)[1]
                correct += pred_indices.eq(batch_label).sum().item()
                for p, l in zip(pred_indices, batch_label):
                    print(f"predicted: {index2region[p.item()]} real:{index2region[l.item()]}")
            
            end_time = time.time()
            print(f"epoch {epoch} time: {end_time-start_time}sec(s).")


            epoch_loss /= len(train_loader)
            self.train_loss.append(epoch_loss)
            epoch_acc = correct / total
            self.train_accuracy.append(epoch_acc)
            print(f"epoch {epoch} train accuracy: {epoch_acc}")
            print(f"epoch {epoch} loss: {epoch_loss}")  


            predicted, labels, val_loss = self.predict(val_loader, device)
            
            val_acc = predicted.eq(labels).sum().item() / len(predicted)
            print(f"epoch {epoch} val accuracy: {val_acc}")

            if val_acc > epoch_acc:
                best_acc = val_acc
                best_epoch = epoch
                torch.save(self.state_dict(), self.best_model_save_path)
            
            self.val_accuracy.append(val_acc)
            self.val_loss.append(val_loss)
            
            
            
        print("Finish!")
        
        return best_acc, best_epoch
            
    def predict(self, test_loader, device):
        self.eval()
        labels = []
        predicted = []
        val_loss = 0.0
        with torch.no_grad():
            for batch_idx, (batch_data, batch_label) in enumerate(tqdm(test_loader)):

                spec, mfcc, chroma = batch_data
                spec, mfcc, chroma = spec.to(device), mfcc.to(device), chroma.to(device)
                batch_data = (spec, mfcc, chroma)
                batch_label = batch_label.to(device)
                
                pred = self.forward(batch_data)

                _, pred_indices = torch.max(pred, axis=1)
                loss = self.loss(pred, batch_label)
                
                val_loss += loss.item()

                predicted.append(pred_indices)
                batch_label = torch.max(batch_label, axis=1)[1]
                labels.append(batch_label)
        val_loss /= len(test_loader)
        predicted = torch.cat(predicted, dim=0)
        labels = torch.cat(labels, dim=0)

        return predicted, labels, val_loss
    
    def plot(self, which):
        
        X = [i for i in range(1, len(self.train_accuracy) + 1)]
        if which=='train_loss':
            y = self.train_loss
        elif which=='train_acc':
            y = self.train_accuracy
        elif which=='val_acc':
            y = self.val_accuracy
        elif which=='val_loss':
            y = self.val_loss

        plt.xlabel("epoch")
        plt.ylabel(which)
        plt.title(which)
        plt.plot(X, y, label="Train loss")
        plt.savefig(f"./model_{which}.png")
        plt.show()

        
        
        

In [42]:
model = MultiModalDialectClassifier().to(device)

In [43]:
train_size = int(len(dataset)*0.8)
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
model.train_(train_loader, test_loader, 0.005, 5, device)

  3%|▎         | 1/33 [00:33<17:43, 33.22s/it]

predicted: jeonla real:gangwon
predicted: jeonla real:jeonla
predicted: jeonla real:chungcheong
predicted: jeonla real:jeju
predicted: gangwon real:chungcheong
predicted: gangwon real:chungcheong
predicted: jeonla real:jeju
predicted: gangwon real:chungcheong
predicted: jeonla real:gangwon
predicted: jeonla real:jeju
predicted: jeju real:chungcheong
predicted: jeonla real:chungcheong
predicted: jeonla real:chungcheong
predicted: jeonla real:chungcheong
predicted: jeonla real:chungcheong
predicted: jeonla real:gangwon
predicted: jeonla real:jeonla
predicted: jeonla real:gyeongsang
predicted: gangwon real:chungcheong
predicted: jeonla real:jeonla
predicted: jeonla real:jeju
predicted: gangwon real:chungcheong
predicted: gangwon real:gangwon
predicted: jeonla real:gangwon
predicted: jeonla real:chungcheong
predicted: jeonla real:jeju
predicted: jeonla real:gangwon
predicted: jeonla real:gyeongsang
predicted: chungcheong real:jeju
predicted: jeonla real:gyeongsang
predicted: jeonla real:je

  6%|▌         | 2/33 [01:07<17:32, 33.96s/it]

predicted: chungcheong real:gangwon
predicted: chungcheong real:jeonla
predicted: chungcheong real:jeju
predicted: chungcheong real:gyeongsang
predicted: chungcheong real:gangwon
predicted: chungcheong real:jeonla
predicted: chungcheong real:chungcheong
predicted: chungcheong real:gyeongsang
predicted: chungcheong real:chungcheong
predicted: chungcheong real:chungcheong
predicted: chungcheong real:chungcheong
predicted: chungcheong real:gyeongsang
predicted: chungcheong real:gangwon
predicted: chungcheong real:gangwon
predicted: chungcheong real:jeonla
predicted: chungcheong real:chungcheong
predicted: chungcheong real:chungcheong
predicted: chungcheong real:gyeongsang
predicted: chungcheong real:chungcheong
predicted: chungcheong real:jeonla
predicted: chungcheong real:jeju
predicted: chungcheong real:gyeongsang
predicted: chungcheong real:gangwon
predicted: chungcheong real:gyeongsang
predicted: chungcheong real:gyeongsang
predicted: chungcheong real:gangwon
predicted: chungcheong re

  9%|▉         | 3/33 [01:43<17:18, 34.61s/it]

...

predicted: chungcheong real:jeonla
predicted: chungcheong real:gyeongsang
predicted: chungcheong real:jeonla
predicted: chungcheong real:jeju
predicted: chungcheong real:gyeongsang
predicted: chungcheong real:gangwon
predicted: chungcheong real:chungcheong
predicted: chungcheong real:jeju
predicted: chungcheong real:chungcheong
predicted: chungcheong real:jeonla
predicted: chungcheong real:gangwon
predicted: chungcheong real:chungcheong
predicted: chungcheong real:jeju
predicted: chungcheong real:gyeongsang
predicted: chungcheong real:jeju
predicted: chungcheong real:gyeongsang
predicted: chungcheong real:jeju
predicted: chungcheong real:gyeongsang
predicted: chungcheong real:chungcheong
predicted: chungcheong real:gangwon
predicted: chungcheong real:chungcheong
predicted: chungcheong real:gyeongsang
predicted: chungcheong real:chungcheong
predicted: chungcheong real:gyeongsang
predicted: chungcheong real:jeju
predicted: chungcheong real:jeonla
predicted: chungcheong real:chungcheong
p

 36%|███▋      | 12/33 [06:20<10:44, 30.71s/it]

predicted: chungcheong real:chungcheong
predicted: chungcheong real:gyeongsang
predicted: chungcheong real:chungcheong
predicted: chungcheong real:chungcheong
predicted: chungcheong real:jeju
predicted: chungcheong real:jeju
predicted: chungcheong real:chungcheong
predicted: chungcheong real:chungcheong
predicted: chungcheong real:jeju
predicted: chungcheong real:jeonla
predicted: chungcheong real:jeonla
predicted: chungcheong real:chungcheong
predicted: chungcheong real:gyeongsang
predicted: chungcheong real:jeju
predicted: chungcheong real:jeju
predicted: chungcheong real:jeonla
predicted: chungcheong real:gangwon
predicted: chungcheong real:gangwon
predicted: chungcheong real:gangwon
predicted: chungcheong real:chungcheong
predicted: chungcheong real:gyeongsang
predicted: chungcheong real:gangwon
predicted: chungcheong real:jeonla
predicted: chungcheong real:jeonla
predicted: chungcheong real:jeonla
predicted: chungcheong real:jeonla
predicted: chungcheong real:chungcheong
predicted

 39%|███▉      | 13/33 [06:51<10:12, 30.64s/it]

predicted: chungcheong real:jeonla
predicted: chungcheong real:jeonla
predicted: chungcheong real:gyeongsang
predicted: chungcheong real:jeonla
predicted: chungcheong real:jeju
predicted: chungcheong real:jeonla
predicted: chungcheong real:gangwon
predicted: chungcheong real:chungcheong
predicted: chungcheong real:chungcheong
predicted: chungcheong real:chungcheong
predicted: chungcheong real:chungcheong
predicted: chungcheong real:jeju
predicted: chungcheong real:jeonla
predicted: chungcheong real:gangwon
predicted: chungcheong real:gangwon
predicted: chungcheong real:gangwon
predicted: chungcheong real:gyeongsang
predicted: chungcheong real:jeonla
predicted: chungcheong real:jeju
predicted: chungcheong real:gyeongsang
predicted: chungcheong real:chungcheong
predicted: chungcheong real:chungcheong
predicted: chungcheong real:jeju
predicted: chungcheong real:jeju
predicted: chungcheong real:gyeongsang
predicted: chungcheong real:gangwon
predicted: chungcheong real:jeju
predicted: chung

In [ ]:
model.plot('train_acc')

In [ ]:
model.plot('train_loss')

In [ ]:
model.plot('val_acc')